In [147]:
import pandas as pd
import stock_analysis
from stock_analysis import StockReader
reader = StockReader('2017-01-01', '2018-12-31')


In [148]:
import pandas as pd
import numpy as np
import yfinance
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 7]
plt.rc('font', size=14)

In [149]:
ticker= yf.Ticker("NFLX")
nflx = ticker.history(interval="1d",start='2017-01-01', end='2018-12-31')


nflx['Date'] = pd.to_datetime(nflx.index)
nflx['Date'] = nflx['Date'].apply(mpl_dates.date2num)
nflx = nflx.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]

import pprint as pp
nflx_info = nflx.info
pp.pprint(nflx_info)


<bound method DataFrame.info of                Date        Open        High         Low       Close
Date                                                               
2017-01-03  17169.0  124.959999  128.190002  124.309998  127.489998
2017-01-04  17170.0  127.489998  130.169998  126.550003  129.410004
2017-01-05  17171.0  129.220001  132.750000  128.899994  131.809998
2017-01-06  17172.0  132.080002  133.880005  129.809998  131.070007
2017-01-09  17175.0  131.479996  131.990005  129.889999  130.949997
...             ...         ...         ...         ...         ...
2018-12-21  17886.0  263.829987  264.500000  241.289993  246.389999
2018-12-24  17889.0  242.000000  250.649994  233.679993  233.880005
2018-12-26  17891.0  233.919998  254.500000  231.229996  253.669998
2018-12-27  17892.0  250.110001  255.589996  240.100006  255.570007
2018-12-28  17893.0  257.940002  261.910004  249.800003  256.079987

[501 rows x 5 columns]>


In [174]:
def isSupport(nflx,i):
  support = nflx['Low'][i] < nflx['Low'][i-1]  and ['Low'][i] < nflx['Low'][i+1] and nflx['Low'][i+1] < nflx['Low'][i+2] and nflx['Low'][i-1] < nflx['Low'][i-2]
  return support
def isResistance(nflx,i):
  resistance = nflx['High'][i] > nflx['High'][i-1]  and nflx['High'][i] > nflx['High'][i+1] and nflx['High'][i+1] > nflx['High'][i+2] and nflx['High'][i-1] > nflx['High'][i-2]
  return resistance




In [175]:
s =  np.mean(nflx['High'] - nflx['Low'])
def isFarFromLevel(l):
   return np.sum([abs(l-x) < s  for x in levels]) == 0

levels = []
for i in range(2,nflx.shape[0]-2):
  if isSupport(nflx,i):
    l = nflx['Low'][i]
    if isFarFromLevel(l):
      levels.append((i,l))
  elif isResistance(nflx,i):
    l = nflx['High'][i]
    if isFarFromLevel(l):
      levels.append((i,l))

IndexError: list index out of range

In [171]:
def plot_all():
  fig, ax = plt.subplots()
  candlestick_ohlc(ax,nflx.values,width=0.6, \
                   colorup='green', colordown='red', alpha=0.8)
  date_format = mpl_dates.DateFormatter('%d %b %Y')
  ax.xaxis.set_major_formatter(date_format)
  fig.autofmt_xdate()
  fig.tight_layout()
  for level in levels:
    plt.hlines(level[1],xmin=nflx['Date'][level[0]],\
               xmax=max(nflx['Date']),colors='blue')
  fig.show()

In [156]:
import itertools

def levels(analyzer, method):
    return [getattr(analyzer, method)(i) for i in range(1, 4)]

# calculate support/resistance levels
nflx_analyzer = stock_analysis.StockAnalyzer(nflx)
support_levels, resistance_levels = (
    levels(nflx_analyzer, metric) for metric in ['support', 'resistance']
)
nflx_viz = stock_analysis.StockVisualizer(nflx)
ax = nflx_viz.evolution_over_time('close', figsize=(15, 8), title='NFLX Closing Price')

for support, resistance, linestyle, level in zip(
    support_levels, resistance_levels, 
    [':', '--', '-.'], itertools.count(1)
):
    nflx_viz.add_reference_line(
        ax, 
        y=support, 
        label=f'support level {level}', 
        color='green', 
        linestyle=linestyle
    )
    nflx_viz.add_reference_line(
        ax, 
        y=resistance, 
        label=f'resistance level {level}', 
        color='red', 
        linestyle=linestyle
    )
ax.get_legend().remove()
ax.set_ylabel('price ($)')

ValueError: DataFrame must contain the following columns: {'high', 'open', 'close', 'low'}

In [27]:
stock_analysis.AssetGroupVisualizer(faang).after_hours_trades()

NameError: name 'faang' is not defined

In [26]:
from stock_analysis.utils import make_portfolio
stock_analysis.StockVisualizer(
    make_portfolio(faang)
).after_hours_trades()

NameError: name 'faang' is not defined

In [6]:
from stock_analysis.utils import make_portfolio
stock_analysis.StockVisualizer(make_portfolio(faang)).open_to_close()

ModuleNotFoundError: No module named 'stock_analysis'

In [25]:
import pandas as pd
import numpy as np

sp_reindexed = sp.reindex(
    pd.date_range('2017-01-01', end='2018-12-31', freq='D')
).assign(
    volume=lambda x: x.fillna(0),
    close=lambda x: x.fillna(method='ffill'),
    open=lambda x: np.where(x.open.isnull(), x.close, x.open),
    high=lambda x: np.where(x.high.isnull(), x.close, x.high),
    low=lambda x: np.where(x.low.isnull(), x.close, x.low)
)

sp_reindexed.index.rename('date', inplace=True)

stock_analysis.StockVisualizer(
    make_portfolio(group_stocks({'bitcoin': bitcoin, 'sp500': sp_reindexed}))
).open_to_close()

NameError: name 'sp' is not defined

In [8]:
fbalx = reader.get_ticker_data('FBALX')
msft = reader.get_ticker_data('MSFT')
mutual_fund = group_stocks({
    '0 - FBALX' : fbalx,
    '1 - Microsoft' : msft,
    '2 - Apple' : aapl,
    '3 - Amazon' : amzn
})
stock_analysis.AssetGroupAnalyzer(mutual_fund).analyze(
    'annualized_volatility'
)

NameError: name 'reader' is not defined

In [9]:
import pandas as pd

def metric_table(stock, index):
    """
    Make a table of metrics for a stock.

    Parameters:
        - stock: The stock's dataframe.
        - index: The dataframe for the index.
     
    Returns:
        A pandas DataFrame with a single row of metrics
    """
    return pd.DataFrame({
        metric : getattr(
            stock_analysis.StockAnalyzer(stock), 
            metric
        )(**kwargs) \
        for metric, kwargs in {
            'alpha' : {'index' : index, 'r_f' : 2.46}, 
            'beta' : {'index' : index}, 
            'annualized_volatility' : {},
            'is_bear_market' : {},
            'is_bull_market' : {}
        }.items()
    }, index=range(1))

metric_table(fbalx, sp)

NameError: name 'fbalx' is not defined